In [1]:
import pandas as pd
from scipy.stats import ranksums
from statsmodels.stats.multitest import multipletests

In [31]:
data_dir = "/Users/cheesemania/PycharmProjects/mscthesis_wrkdir/metadata"

In [32]:
data_classical = pd.read_csv(f'{data_dir}/combined_alpha_diversity_rar3000.tsv', sep='\t')
data_q2_boots = pd.read_csv(f'{data_dir}/combined_alpha_diversity_q2_boots_rar3000_n50.tsv', sep='\t')

In [33]:
# Function to perform Wilcoxon rank-sum test (equivalent to the Kruskal-Wallis test for binary data) 
# and return p-values
def wilcoxon_test_within_project(project_data, feature_col):
    cases = project_data[project_data['Case_Control'] == 'Case'][feature_col]
    controls = project_data[project_data['Case_Control'] == 'Control'][feature_col]
    
    if len(cases) > 0 and len(controls) > 0:
        stat, p_value = ranksums(cases, controls)
        return p_value
    else:
        return None

In [34]:
# Function to calculate percentage change between cases and controls
# Median or mean? - Mean
def calculate_percentage_change(project_data, feature_col):
    cases_mean = project_data[project_data['Case_Control'] == 'Case'][feature_col].mean()
    controls_mean = project_data[project_data['Case_Control'] == 'Control'][feature_col].mean()
    
    if controls_mean != 0:  # Avoid division by zero
        percentage_change = ((cases_mean - controls_mean) / controls_mean) * 100
        return percentage_change
    else:
        return None

In [35]:
# Classical alpha diversity
comparison_cols = ['observed_features_non_filtered', 'observed_features_filt_nonresident',
                   'observed_features_nonresident', 'shannon_entropy_nonresident',
                   'shannon_entropy_non_filtered', 'shannon_entropy_filt_nonresident']

# Create a DataFrame to store results
results = []

# Iterate over each project in data_classical
for project_id, project_data in data_classical.groupby('Project_ID'):
    for col in comparison_cols:
        p_value = wilcoxon_test_within_project(project_data, col)
        percentage_change = calculate_percentage_change(project_data, col)
        results.append({
            'project_id': project_id, 
            'comparison': col, 
            'p_value': p_value, 
            'percentage_change': percentage_change
        })

# Convert results into a DataFrame
classical_results_df = pd.DataFrame(results)

# Perform multiple testing correction (FDR)
classical_results_df['q_value'] = multipletests(classical_results_df['p_value'], method='fdr_bh')[1]

In [36]:
classical_results_df

,project_id,comparison,p_value,percentage_change,q_value
0,PRJDB13192,observed_features_non_filtered,0.014416,-27.034909,0.064874
1,PRJDB13192,observed_features_filt_nonresident,0.006035,-31.548999,0.030416
2,PRJDB13192,observed_features_nonresident,0.111392,-3.561644,0.264201
3,PRJDB13192,shannon_entropy_nonresident,0.781171,139.751244,0.911521
4,PRJDB13192,shannon_entropy_non_filtered,0.000345,-52.439817,0.006202
...,...,...,...,...,...
121,crohns-paper_37122605,observed_features_filt_nonresident,0.000005,-32.163621,0.000201
122,crohns-paper_37122605,observed_features_nonresident,0.017221,-20.200000,0.065753
123,crohns-paper_37122605,shannon_entropy_nonresident,0.016595,-30.189085,0.065753
124,crohns-paper_37122605,shannon_entropy_non_filtered,0.583375,-4.197466,0.802082


In [37]:
# q2-boots alpha diversity
comparison_cols = ['observed_features_non_filtered', 'observed_features_filt_nonresident',
                   'observed_features_nonresident', 'shannon_entropy_nonresident',
                   'shannon_entropy_non_filtered', 'shannon_entropy_filt_nonresident']

# Create a DataFrame to store results
results = []

# Iterate over each project in data_classical
for project_id, project_data in data_q2_boots.groupby('Project_ID'):
    for col in comparison_cols:
        p_value = wilcoxon_test_within_project(project_data, col)
        percentage_change = calculate_percentage_change(project_data, col)
        results.append({
            'project_id': project_id, 
            'comparison': col, 
            'p_value': p_value, 
            'percentage_change': percentage_change
        })

# Convert results into a DataFrame
q2_boots_results_df = pd.DataFrame(results)

# Perform multiple testing correction (FDR)
q2_boots_results_df['q_value'] = multipletests(q2_boots_results_df['p_value'], method='fdr_bh')[1]

In [38]:
q2_boots_results_df

,project_id,comparison,p_value,percentage_change,q_value
0,PRJDB13192,observed_features_non_filtered,0.017696,-23.726027,0.082580
1,PRJDB13192,observed_features_filt_nonresident,0.035307,-26.910070,0.158883
2,PRJDB13192,observed_features_nonresident,0.451302,-52.179328,0.900867
3,PRJDB13192,shannon_entropy_nonresident,0.556775,-26.548297,0.900867
4,PRJDB13192,shannon_entropy_non_filtered,0.000389,-52.489603,0.008238
...,...,...,...,...,...
121,crohns-paper_37122605,observed_features_filt_nonresident,0.000005,-31.701995,0.000680
122,crohns-paper_37122605,observed_features_nonresident,0.114185,490.625000,0.358813
123,crohns-paper_37122605,shannon_entropy_nonresident,0.159977,656.163245,0.447936
124,crohns-paper_37122605,shannon_entropy_non_filtered,0.643838,-3.820081,0.922139


In [51]:
# Aggregate the original data to get unique project-level data
aggregated_data_classical = data_classical.groupby('Project_ID').agg({
    'Disease_Level_1': 'first',  # each project has a single value for these columns
    'Disease_Level_2': 'first' 
}).reset_index()

aggregated_data_boots = data_q2_boots.groupby('Project_ID').agg({
    'Disease_Level_1': 'first',  # each project has a single value for these columns
    'Disease_Level_2': 'first' 
}).reset_index()

In [53]:
classical_results_df_with_disease = pd.merge(classical_results_df, aggregated_data_classical[['Project_ID', 'Disease_Level_1', 'Disease_Level_2']], left_on='project_id', right_on='Project_ID', how='left')

q2_boots_results_df_with_disease = pd.merge(q2_boots_results_df, aggregated_data_boots[['Project_ID', 'Disease_Level_1', 'Disease_Level_2']], left_on='project_id', right_on='Project_ID', how='left')

In [55]:
classical_results_df_with_disease = classical_results_df_with_disease.drop(columns=['Project_ID'])

q2_boots_results_df_with_disease = q2_boots_results_df_with_disease.drop(columns=['Project_ID'])

In [56]:
# Save the results to a new TSV file
classical_results_df_with_disease.to_csv(f'{data_dir}/classical_alpha_stats_for_plots.tsv', sep='\t', index=False)
q2_boots_results_df_with_disease.to_csv(f'{data_dir}/boots_alpha_stats_for_plots.tsv', sep='\t', index=False)

### Subsetting the dataframe (only the classical alpha diversity)

In [58]:
# Non-filtered
non_filtered_results_df = classical_results_df_with_disease[
    classical_results_df_with_disease['comparison'].isin(['observed_features_non_filtered', 'shannon_entropy_non_filtered'])
]

In [59]:
# Filt-nonresident
filt_nonresident_results_df = classical_results_df_with_disease[
    classical_results_df_with_disease['comparison'].isin(['observed_features_filt_nonresident', 'shannon_entropy_filt_nonresident'])
]

In [60]:
# Non-resident
nonresident_results_df = classical_results_df_with_disease[
    classical_results_df_with_disease['comparison'].isin(['observed_features_nonresident', 'shannon_entropy_nonresident'])
]

In [61]:
non_filtered_results_df.to_csv(f'{data_dir}/non_filtered_alpha_stats_for_plots.tsv', sep='\t', index=False)
filt_nonresident_results_df.to_csv(f'{data_dir}/filt_nonresident_alpha_stats_for_plots.tsv', sep='\t', index=False)
nonresident_results_df.to_csv(f'{data_dir}/nonresident_alpha_stats_for_plots.tsv', sep='\t', index=False)

#### Sanity check

In [50]:
print(classical_results_df_with_disease['Disease_Level_1'].unique())

['Infectious' 'Cancer' 'Other' 'Gastrointestinal' 'Liver' 'Metabolic'
 'Autoimmune']


In [48]:
print(classical_results_df_with_disease['Disease_Level_2'].unique())

['COVID-19' 'Melanoma' 'AN' 'IBS' 'HIV' 'CDI' 'Alcohol-Associated-LD'
 'IBD' 'CKD' 'T2DM' 'NAFLD' 'RA' 'AD' 'T1DM' 'HD' 'GDM']
